In [38]:
import joblib
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
import glob

In [39]:
# Read positive embeddings
positive_embeddings = []
for file_path in glob.glob('face_embeddings/*.npy'):
    embedding = np.load(file_path)
    positive_embeddings.append(embedding)

# Read negative embeddings
negative_data = []
for file_path in glob.glob('neg_embeddings/*.npy'):
    embedding = np.load(file_path)
    # embedding = np.resize(embedding, (1, 8631)) # Resize to 1x8631
    negative_data.append(embedding)

positive_embeddings = np.array(positive_embeddings)

negative_data = np.array(negative_data)

In [40]:
print(positive_embeddings.squeeze().shape)
print(negative_data.squeeze().shape)
positive_embeddings = positive_embeddings.squeeze()
negative_data = negative_data.squeeze()


(264, 8631)
(793, 8631)


### Create X and y data sets with proper labels for y

In [41]:
# Reshape negative data to match face embeddings shape
# negative_data = negative_data.reshape((positive_embeddings.shape[0], -1))

# Create labels for face embeddings (1) and negative data (0)
face_labels = np.ones((positive_embeddings.shape[0],))
negative_labels = np.zeros((negative_data.shape[0],))

# Combine face embeddings and negative data
X = np.concatenate((positive_embeddings, negative_data), axis=0)
y = np.concatenate((face_labels, negative_labels), axis=0)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
# Train SVM model
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)

# Evaluate model on testing set
score = clf.score(X_test, y_test)
print('Accuracy:', score)


Accuracy: 1.0


In [43]:
filename = 'svm_model.pkl'
joblib.dump(clf, filename)

['svm_model.pkl']